In [1]:
from llama_cpp import Llama
import pandas as pd
import os
import json
import time
import re
from docx import Document
import subprocess
from tqdm import tqdm
from pathlib import Path
import string
import random
from collections import defaultdict
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
# Используемые llm 
llms = [
        ("Mistral", Llama(model_path="./IMPORTS/mistral-7b-instruct-v0.1.Q4_K_M.gguf", n_ctx=2048, verbose=False)),
]
folder = "./data"

llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


In [3]:
# Если файл состоит из нескольких страниц, разделяет файл на несколько по одной странице
def split_files(input_folder, output_folder=None):
    if output_folder is None:
        output_folder = os.path.join(input_folder, "split")
        
    os.makedirs(output_folder, exist_ok=True)
    result = defaultdict(list)
    
    for file in sorted(os.listdir(input_folder)):
        
        file_path = os.path.join(input_folder, file)
        file_name, file_ext = os.path.splitext(file)
        
        if file_ext.lower() == '.xlsx':
            try:
                sheets = pd.read_excel(file_path, sheet_name=None)
                for sheet_name, df in sheets.items():
                    output_name = f"{file_name}_{sheet_name}.xlsx"
                    output_path = os.path.join(output_folder, output_name)
                    df.to_excel(output_path, index=False)
                    result[os.path.basename(file_path)].append(output_path)  
                    
            except Exception as e:
                print(f"Error processing Excel file {file}: {e}")

        elif file_ext.lower() == '.docx':
            try:
                doc = Document(file_path)
                if len(doc.paragraphs) > 0:
                    output_path = os.path.join(output_folder, file)
                    doc.save(output_path)
                    result[os.path.basename(file_path)].append(output_path)
                    
            except Exception as e:
                print(f"Error processing Word file {file}: {e}")

    return dict(result) 

In [4]:
# Перевод docx в pdf
def docx_to_pdf(docx_path, output_dir=None):
    if output_dir is None:
        output_dir = os.path.dirname(docx_path) or "."

    subprocess.run([
        "libreoffice", "--headless", "--convert-to", "pdf",
        docx_path, "--outdir", output_dir
    ], check=True)

    pdf_name = os.path.splitext(os.path.basename(docx_path))[0] + ".pdf"
    pdf_path = os.path.join(output_dir, pdf_name)

    return pdf_path

In [5]:
# Функция нормализации строки из таблицы
def normalize_row_string(row):
    processed_cells = []
    for cell in row:
        if cell and str(cell).strip():
            cell_str = str(cell).strip().lower()
            punctuation_chars = string.punctuation + '«»—–'
            translator = str.maketrans('', '', punctuation_chars)
            clean_cell = cell_str.translate(translator)
            processed_cells.append(clean_cell)    
    row_str = " ".join(processed_cells)
    clean_str = re.sub(r'\s+', ' ', row_str).strip()
    return clean_str

In [6]:
# Функция проверки строки
def check_info_row(llm, row, previous_row = None, previous_row_type = None):
    try:
        row_str = normalize_row_string(row)

        header_keywords = ["наименование", "название", "адрес", "страховая", "доступ", 
                          "программа", "лпу", "вид", "помощи"]
        if len([x for x in row if len(x)>0]) < 2:
            return "empty", 1    
        
        found_keywords = {keyword for keyword in header_keywords if keyword in row_str}
        if len(found_keywords) >= 2 and len(row_str.split()) > 5 and len(row_str.split())/len(row) <= 2:
                return "header", 1

        address_indicators = r'\b(ООО|ул|улица|пр-кт|просп|проспект|пер|переулок|ш|шоссе|наб|набережная|пл|площадь|д[\dа-я]*|дом|корп|к[\dа-я]*|строение|стр[\dа-я]*)\b'

        has_address = re.search(address_indicators, row_str, re.IGNORECASE)
        if has_address:
            return "info", 1
        if previous_row_type == "info" and previous_row:
            prev_non_empty = len([x for x in previous_row if x and str(x).strip()])
            curr_non_empty = len([x for x in row if x and str(x).strip()])        
            if curr_non_empty == prev_non_empty and curr_non_empty > 1:
                return "info", 1


        row_text = " | ".join(str(cell) for cell in row if pd.notna(cell))
        if not row_text.strip():
            return "empty", 1
        prompt = f"""
        Определи тип строки из таблицы. Варианты:
        - "header" - если это заголовки столбцов (например, содержат слова: название, адрес, город и т.п.)
        - "info" - если это данные (например, содержат конкретные значения: названия организаций, адреса)
        - "empty" - если строка не содержит полезной информации (шапка таблицы, пустые значения)

        Ответ должен быть только одним словом: "header", "info" или "empty".

        Пример header:
        Название | Адрес | Вид | помощи | №1 | №2 | №3

        Пример info:
        СПб ГУЗ "Городская поликлиника №51" | СПб, пр. Космонавтов, д.35 | ПВ | + | + | +

        Пример empty: 
        Амбулаторно-поликлиническая помощь | | |  

        Строка для анализа: "{row_text}" 
        """
        response = llm.create_chat_completion(
                messages=[{"role": "user", "content": prompt}]
            )
        result = response["choices"][0]["message"]["content"].strip().lower()
#         print(f"ROW: {row}")
#         print(f"RESULT: {result}")
        if "header" in result:
            if len(found_keywords) >= 1:
                return "header", 0
            else:
                return "info", 0
        elif "info" in result:
            return "info", 0
        else:
            return "empty", 0
    except:
        return "empty", 0

In [7]:
def find_headers(llm, row_headers):
    s = ", ".join(row_headers)
    prompt = f"""
    Ты — эксперт по структурированию медицинских данных. Найди точные соответствия между предоставленными заголовками и эталонными.
    Ответ должен быть только одним JSON

    Эталонные заголовки (используй ТОЛЬКО эти):
    "Название поликлиники"
    "Адрес"
    "Тип доступа"
    "Виды помощи"

    Строгие правила:
    1. Сопоставляй ТОЛЬКО если уверен на 100%
    2. Для "Название поликлиники" принимай:
       - "ЛПУ", "Медцентр", "Клиника", "Больница"
    3. Для "Адрес" ищи:
       - "Местоположение", "Адрес", "Локация"
    4. "Тип доступа":
       - "Доступ", "Обслуживание", "Условия"
    5. Для "Виды помощи" ищи:
       - "Виды обращений", "Вид помощи", "Тип помощи", "Риск", "Вид медицинской помощи"

    исходный заголовок может быть ТОЛЬКО из эталонных заголовков:
    "Название поликлиники"
    "Адрес"
    "Тип доступа"
    "Виды помощи"

   Пример 1: 
   Наименование, Адрес(фактич.), Вид помощи, №1, №2, №3, Доступ
   Ответ:
   {{
    "Название поликлиники": "Наименование",
    "Адрес": "Адрес(фактич.)",
    "Тип доступа": "Доступ",
    "Виды помощи": "Вид помощи"
   }}

   Пример 2: 
   Филиал, ЛПУ, Адрес ЛПУ, Риск, Доступ в ЛПУ
   Ответ:
   {{
    "Название поликлиники": "ЛПУ",
    "Адрес": "Адрес ЛПУ",
    "Страховая компания": "-",
    "Тип доступа": "Доступ в ЛПУ",
    "Виды помощи": "Риск"
   }}

    Предоставленные заголовки для анализа, использовать можно ТОЛЬКО их:
    {s}
    Ответ должен быть только одним JSON
    """        
    response = llm.create_chat_completion(
        messages=[{"role": "user", "content": prompt}]
    )
    result = response["choices"][0]["message"]["content"]
    print(f"ROW: {row_headers}")
    print(f"RESULT: {result}")
    json_str = result[result.find("{"):result.rfind("}")+1]
    json_str = json_str.replace("'", '"')

    mapping = json.loads(json_str)

    return mapping

In [8]:
def check_info_row_llm(llm, row):
    try:
        row_text = " | ".join(str(cell) for cell in row if pd.notna(cell))
        if not row_text.strip():
            return "empty"
        prompt = f"""
        Определи тип строки из таблицы. Варианты:
        - "header" - если это заголовки столбцов (например, содержат слова: название, адрес, город и т.п.)
        - "info" - если это данные (например, содержат конкретные значения: названия организаций, адреса)
        - "empty" - если строка не содержит полезной информации (шапка таблицы, пустые значения)

        Ответ должен быть только одним словом: "header", "info" или "empty".

        Пример header:
        Название | Адрес | Вид | помощи | №1 | №2 | №3

        Пример info:
        СПб ГУЗ "Городская поликлиника №51" | СПб, пр. Космонавтов, д.35 | ПВ | + | + | +

        Пример empty: 
        Амбулаторно-поликлиническая помощь | | |  
     
        Строка для анализа: "{row_text}" 
        """
        response = llm.create_chat_completion(
                messages=[{"role": "user", "content": prompt}]
            )
        result = response["choices"][0]["message"]["content"].strip().lower()
        if "header" in result:
            return "header"
        elif "info" in result:
            return "info"
        else:
            return "empty"
    except:
        return "empty"

In [9]:
standard_headers = [
    "Название поликлиники",
    "Адрес",
    "Виды помощи",
    "Тип доступа",
    "Страховая компания",
]

standard_val_headers = [
    "Row",
    "Pred_type",
    "Type",
]

standard_second_val_headers = [
    "Row",
    "Mapping",
]

files = split_files(folder)
dict_to_standard = {}  # {"Стандартное название" : "Нестандартное название"}

global_df = pd.DataFrame(columns=standard_headers)

test_val_heuristic = pd.DataFrame(columns=standard_val_headers)
test_val_llm = pd.DataFrame(columns=standard_val_headers)

test_second_query = pd.DataFrame(columns=standard_second_val_headers)

for model_name, llm in llms:
    print(f"\nМодель: {model_name}")
    start_time = time.time()
    for file_name, pages in files.items():
        name_company = os.path.splitext(file_name)[0]
        if os.path.splitext(file_name)[1] == '.docx':
            continue
        standard_df = pd.DataFrame(columns=standard_headers)
            
        for file in tqdm(pages, desc="Обработка файлов", unit="file"):
                
            output_dir = Path(f"Merged_tables")
            output_dir.mkdir(parents=True, exist_ok=True)
            
            
            df = pd.read_excel(os.path.join(file), header = None)
            n = 0
            current_headers = []
            prev_row, prev_info = None, None
            
            
            for index, row in tqdm(
                df.iterrows(), 
                total=len(df),
                desc=f"Строки ({os.path.basename(file)})",
                unit="строка",
                leave=False
            ):
                row_data = row.values.tolist()
                row_data = [value if pd.notna(value) else "" for value in row.values]

                info, flag = check_info_row(llm, row_data, prev_row, prev_info)
                
                val_row = {"Row": row_data, "Pred_type": info}
                if flag == 1:
                    test_val_heuristic = pd.concat([test_val_heuristic, pd.DataFrame([val_row])], ignore_index=True)
                else:
                    test_val_llm = pd.concat([test_val_llm, pd.DataFrame([val_row])], ignore_index=True)                            
                
                prev_info, prev_row = info, row_data
    
#                 if n < 20:
#                     print(f"ROW {row_data}")
#                     print(f"INFO {info}")
                n += 1

                if info == "header":
                    current_headers = row_data
                    header_mapping = find_headers(llm, row_data)
                    
                    val_row = {"Row": row_data, "Mapping": header_mapping}
                    test_second_query = pd.concat([test_second_query, pd.DataFrame([val_row])], ignore_index=True)                            
                    
                    for standard, non_standard in header_mapping.items():
                        if standard in standard_headers:
                            dict_to_standard[non_standard] = standard 
                    
                elif info == "info":
                    new_row = {col: "" for col in standard_headers}
                    for col_idx, value in enumerate(row_data):
                        if col_idx >= len(current_headers):
                            continue
                        data_header = current_headers[col_idx]
                        standard_header = dict_to_standard.get(data_header)
                        if standard_header and standard_header in new_row:
                            new_row[standard_header] = value

                    if any(new_row.values()):
                        new_row["Страховая компания"] = name_company
                        standard_df = pd.concat([standard_df, pd.DataFrame([new_row])], ignore_index=True)
                        global_df = pd.concat([global_df, pd.DataFrame([new_row])], ignore_index=True)
                elif info == "empty":
                    continue
            
            
        standard_df.to_excel(output_dir / f"merged_{name_company}.xlsx", index=False)

           
    random.seed(42)
    sampled_rows = test_val_heuristic.sample(n=100, random_state=42)

    test_bonus_val_llm = test_val_llm
    for _, row in sampled_rows.iterrows():
        new_row = {
            "Row": row["Row"],
            "Pred_type": check_info_row_llm(llm, row["Row"]),
        }
        test_bonus_val_llm = pd.concat([test_bonus_val_llm, pd.DataFrame([new_row])], ignore_index=True)                            

        
    global_df.to_excel(output_dir / f"merged_global.xlsx", index=False)

    output_dir_stats = Path(f"Stats")
    output_dir_stats.mkdir(parents=True, exist_ok=True)

    test_val_heuristic.to_excel(output_dir_stats / f"test_val_heuristic.xlsx", index=False)
    test_val_llm.to_excel(output_dir_stats / f"test_val_llm.xlsx", index=False)
    test_bonus_val_llm.to_excel(output_dir_stats / f"test_bonus_val_llm.xlsx", index=False)

    test_second_query.to_excel(output_dir_stats / f"test_second_query.xlsx", index=False)

            
    elapsed_time = time.time() - start_time
    print(f"\n Модель {model_name} завершена!")
    print(f"Затраченное время: {elapsed_time:.2f} сек")


Модель: Mistral


Строки (Альфа(1)_ЛПУ_СПб.xlsx):   3%|▏     | 34/975 [01:49<33:15,  2.12s/строка]

ROW: ['Название', 'Адрес', 'Вид помощи', '№1', '№2', '№3', '', '']
RESULT:  {
    "Название поликлиники": "Название",
    "Адрес": "Адрес",
    "Тип доступа": "Доступ",
    "Виды помощи": "Вид помощи"
  }



Строки (Альфа(1)_Прямой доступ_СПб.xlsx):   0%|     | 0/117 [00:00<?, ?строка/s]
Строки (Альфа(1)_Прямой доступ_СПб.xlsx):   3%| | 4/117 [00:17<08:08,  4.32s/стр
Строки (Альфа(1)_Прямой доступ_СПб.xlsx):  21%|▏| 24/117 [00:17<00:49,  1.87стро

ROW: ['Название', 'Адрес', 'Вид помощи', '№1', '№2', '№3']
RESULT:  {
    "Название поликлиники": "Наименование",
    "Адрес": "Адрес",
    "Тип доступа": "Доступ",
    "Виды помощи": "Вид помощи"
  }


...

ROW: ['Город', '№ Программы', 'Наименование ЛПУ', 'Адрес ЛПУ', 'Виды помощи']
RESULT:  {
    "Название поликлиники": "Наименование ЛПУ",
    "Адрес": "Адрес ЛПУ",
    "Тип доступа": "-",
    "Виды помощи": "Виды помощи"
  }



Строки (Согласие_5 Новосибирск.xlsx):  11%| | 18/170 [00:15<01:17,  1.97строка/s
Строки (Согласие_5 Новосибирск.xlsx):  16%|▏| 27/170 [00:15<00:40,  3.56строка/s
Строки (Согласие_5 Новосибирск.xlsx):  22%|▏| 37/170 [00:15<00:22,  5.96строка/s
Строки (Согласие_5 Новосибирск.xlsx):  28%|▎| 47/170 [00:16<00:13,  9.18строка/s
Строки (Согласие_5 Новосибирск.xlsx):  34%|▎| 57/170 [00:16<00:08, 13.30строка/s
Строки (Согласие_5 Новосибирск.xlsx):  39%|▍| 66/170 [00:16<00:05, 18.00строка/s
Строки (Согласие_5 Новосибирск.xlsx):  44%|▍| 75/170 [00:16<00:03, 23.78строка/s
Строки (Согласие_5 Новосибирск.xlsx):  49%|▍| 84/170 [00:16<00:02, 29.65строка/s
Строки (Согласие_5 Новосибирск.xlsx):  55%|▌| 93/170 [00:16<00:02, 36.38строка/s
Строки (Согласие_5 Новосибирск.xlsx):  59%|▌| 101/170 [00:16<00:01, 42.64строка/
Строки (Согласие_5 Новосибирск.xlsx):  64%|▋| 109/170 [00:16<00:01, 48.29строка/
Строки (Согласие_5 Новосибирск.xlsx):  70%|▋| 119/170 [00:16<00:00, 57.51строка/
Строки (Согласие_5 Новосиби

ROW: ['Программа', 'Город ЛПУ', 'Наименование ЛПУ', 'Адрес ЛПУ', 'Виды обращений']
RESULT:  {
    "Название поликлиники": "Наименование ЛПУ",
    "Адрес": "Адрес ЛПУ",
    "Тип доступа": "Доступ в ЛПУ",
    "Виды помощи": "Виды обращений"
  }



Обработка файлов: 100%|█████████████████████████| 2/2 [01:33<00:00, 46.69s/file]



 Модель Mistral завершена!
Затраченное время: 2210.59 сек
